In [1]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from ultralytics.utils.downloads import download
import yaml

In [2]:
# import yaml file
with open('VisDrone.yaml', 'r') as file:
    yaml_data = yaml.safe_load(file)

# YAML verisinden 'path' anahtarını kullanarak bir yol belirleyin
dir = Path(yaml_data['path']) 


In [3]:
# Download data
dir = Path(yaml_data['path'])  # dataset root dir
urls = ['https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-train.zip',
      'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-val.zip',
      'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-dev.zip',
      'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-challenge.zip']
download(urls, dir=dir, curl=True, threads=4)


In [4]:
# converting algorithm
def visdrone2yolo(dir):

    def convert_box(size, box):
      # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    (dir / 'labels').mkdir(parents=True, exist_ok=True)  # make labels directory
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # VisDrone 'ignored regions' class 0
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                with open(str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}'), 'w') as fl:
                    fl.writelines(lines)  # write label.txt



In [5]:
# Converting process
dir = Path(yaml_data['path']) 
for d in 'VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev':
    visdrone2yolo(dir / d)  # convert VisDrone annotations to YOLO labels

Converting ..\VisDrone\VisDrone2019-DET-train: 6471it [00:56, 115.16it/s]
Converting ..\VisDrone\VisDrone2019-DET-val: 548it [00:06, 84.41it/s] 
Converting ..\VisDrone\VisDrone2019-DET-test-dev: 1610it [00:12, 131.38it/s]
